# Title: ...

## 0. Imports

In [145]:
# Notebook similar to: https://www.kaggle.com/code/gracehephzibahm/prediction-of-rent-prices-in-barcelona
# Web scraper with good insights and EDA: https://github.com/agonzalezramos/Idealista-Price-Prediction/tree/main

In [146]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import geopandas as gpd

## 1. Load Data

In [147]:
# Load Data
og_df = pd.read_csv('../data/pisos_clean.csv', sep=',')

C:\Users\pdpau\AppData\Local\Temp\ipykernel_24492\4198264111.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  og_df = pd.read_csv('../data/pisos_clean.csv', sep=',')


## 2. Data Preparation

In [148]:
df = og_df.copy()

df.head(5)

summary               location      price  \
0      Casa en calle Urb. Las Mimosas        Ames (San Tome)  250.000 €   
1                  Piso en Milladoiro        Ames (San Tome)   90.000 €   
2             Casa en calle La Piedra  A Ortigueira (Cariño)   50.000 €   
3                     Casa en Perillo      Perillo (Oleiros)  359.000 €   
4  Chalet en Urbanización Las Mimosas    O Milladoiro (Ames)  200.430 €   

     size rooms    price/m2 bathrooms Num Photos   type    region  
0  315 m²     4    793 €/m²         3         31  pisos  a_coruna  
1   72 m²     2  1.250 €/m²         1         11  pisos  a_coruna  
2   92 m²     2    543 €/m²         1         40  pisos  a_coruna  
3  244 m²     4  1.471 €/m²         3         39  pisos  a_coruna  
4  315 m²     4    636 €/m²         3         24  pisos  a_coruna

In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 777179 entries, 0 to 777178
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   summary     777179 non-null  object
 1   location    776886 non-null  object
 2   price       776898 non-null  object
 3   size        776892 non-null  object
 4   rooms       752352 non-null  object
 5   price/m2    744608 non-null  object
 6   bathrooms   752526 non-null  object
 7   Num Photos  776898 non-null  object
 8   type        776898 non-null  object
 9   region      776282 non-null  object
dtypes: object(10)
memory usage: 59.3+ MB


In [150]:
# Drop NaN values
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 719459 entries, 0 to 777178
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   summary     719459 non-null  object
 1   location    719459 non-null  object
 2   price       719459 non-null  object
 3   size        719459 non-null  object
 4   rooms       719459 non-null  object
 5   price/m2    719459 non-null  object
 6   bathrooms   719459 non-null  object
 7   Num Photos  719459 non-null  object
 8   type        719459 non-null  object
 9   region      719459 non-null  object
dtypes: object(10)
memory usage: 60.4+ MB


## 3. Exploratory Data Analysis

In [151]:
df.tail(5)

summary  \
777170        Finca rústica en Carrer Bosc, nº 51   
777171              Finca rústica en Els Dominics   
777176        Finca rústica en La Roca del Vallès   
777177  Finca rústica en Bajada Les Valls, nº S/N   
777178                     Loft en calle Catlunya   

                                   location        price      size rooms  \
777170                        Lliçà d'Amunt  1.100.000 €  1.120 m²    10   
777171                             Cardedeu  1.500.000 €    520 m²     8   
777176                   La Roca del Vallès  1.350.000 €    350 m²     5   
777177                   La Roca del Vallès    456.000 €    202 m²     5   
777178  Llerona (Les Franqueses del Vallès)    155.000 €     87 m²     1   

          price/m2 bathrooms Num Photos             type               region  
777170    982 €/m²       1.0         32  fincas_rusticas  valles_oriental_sud  
777171  2.884 €/m²       3.0         12  fincas_rusticas  valles_oriental_sud  
777176  3.857 €/m²       2.0          9  fincas_rusticas  valles_oriental_sud  
777177  2.257 €/m²       3.0         30  fincas_rusticas  valles_oriental_sud  
777178  1.781 €/m²       1.0         34            lofts  valles_oriental_sud

### 3.1 Dataset compraventa

In [152]:
# TODO: Explore by location

In [153]:
df['region'].value_counts()

df_counts = df[df['region'].str.contains('sabadell')]

df_counts['region'].value_counts()

region
sabadell    2347
Name: count, dtype: int64

In [92]:
# Filtrar nomes per Catalunya

# value counts de region
df['region'].value_counts()

# Filtrar nomes per Barcelona
df_barna = df[df['region'] == 'barcelona']
df_barna

summary  \
20983   Piso en Carrer del Bruc, cerca de Carretera de...   
20984                  Piso en Carrer de les Orenetes, 37   
20985   Casa en Carrer de la Creu, cerca de Carrer de ...   
20986   Piso en Centre-Zona Estació-Passeig de Joan Ma...   
20987                         Piso en Carrer del Retir, 4   
...                                                   ...   
269325           Loft en Carrer de Francisco Giner, nº 40   
269326                Loft en Carrer de Ramón y Cajal, 58   
269327                   Loft en Carrer d'Andrade, nº 218   
269330                                  Loft en Barcelona   
269331                   Loft en Carrer de l'Harmonia, 35   

                                                 location        price  \
20983                 Valldaura-Plaça Catalunya (Manresa)     85.000 €   
20984                                       Viladecavalls    149.900 €   
20985                                   Sant Just Desvern  1.095.000 €   
20986   Centre-Zona Estació-Passeig de Joan Maragall (...    297.000 €   
20987                                Castellar del Vallès    230.000 €   
...                                                   ...          ...   
269325  La Vila de Gràcia (Distrito Gràcia. Barcelona ...    380.000 €   
269326  La Vila de Gràcia (Distrito Gràcia. Barcelona ...    220.000 €   
269327  Sant Martí de Provençals (Distrito Sant Martí....    113.000 €   
269330  Les Corts (Distrito Les Corts. Barcelona Capital)    419.300 €   
269331  Montbau (Distrito Horta-Guinardó. Barcelona Ca...     89.900 €   

          size rooms    price/m2 bathrooms Num Photos   type     region  
20983    87 m²     4    977 €/m²         1         10  pisos  barcelona  
20984    57 m²     2  2.629 €/m²         1         16  pisos  barcelona  
20985   371 m²     6  2.951 €/m²         4         46  pisos  barcelona  
20986   112 m²     3  2.651 €/m²         2         30  pisos  barcelona  
20987    70 m²     3  3.285 €/m²         1          1  pisos  barcelona  
...        ...   ...         ...       ...        ...    ...        ...  
269325  180 m²     2  2.111 €/m²       2.0         11  lofts  barcelona  
269326   87 m²     3  2.528 €/m²       1.0          8  lofts  barcelona  
269327   55 m²     1  2.054 €/m²       1.0          9  lofts  barcelona  
269330   93 m²     1  4.508 €/m²       1.0         20  lofts  barcelona  
269331   60 m²     2  1.498 €/m²       1.0         17  lofts  barcelona  

[9883 rows x 10 columns]

In [103]:
# Filtrar si location contiene Manresa
df_barris_barna = df_barna[df_barna['location'].str.contains('Barcelona')]
df_barris_barna['location'].value_counts()

location
La Dreta de l'Eixample (Distrito Eixample. Barcelona Capital)             110
El Raval (Distrito Ciutat Vella. Barcelona Capital)                        86
La Nova Esquerra de l'Eixample (Distrito Eixample. Barcelona Capital)      71
L'Antiga Esquerra de l'Eixample (Distrito Eixample. Barcelona Capital)     70
Sant Gervasi-Galvany (Distrito Sarrià-Sant Gervasi. Barcelona Capital)     69
                                                                         ... 
La Vall d'Hebron (Distrito Horta-Guinardó. Barcelona Capital)               3
Ciutat Meridiana (Distrito Nou Barris. Barcelona Capital)                   2
La Clota (Distrito Horta-Guinardó. Barcelona Capital)                       1
Baró de Viver (Distrito Sant Andreu. Barcelona Capital)                     1
Nou Barris (Barcelona Capital)                                              1
Name: count, Length: 70, dtype: int64

### 3.2 Index preu dels lloguers per municipi 2022 (es podria fer per districte de Barna)

In [ ]:
# Fuente: https://www.mivau.gob.es/vivienda/alquila-bien-es-tu-derecho/serpavi
# https://www.kaggle.com/datasets/alelafe/rent-prices-spain

' # Carga el archivo SHP\nshapefile = gpd.read_file("../data/ALQ/ALQ_Municipios_2022_Web.shp")\n\n# Guarda los datos en un archivo CSV\nshapefile.to_csv("archivo.csv", index=False) '

In [95]:
a1 = pd.read_csv("archivo1.csv", sep=',')
a2 = pd.read_csv("archivo2.csv", sep=',')
a3 = pd.read_csv("archivo3.csv", sep=',')
a4 = pd.read_csv("archivo4.csv", sep=',')

new_df = pd.concat([a1, a2, a3, a4], ignore_index=True)

In [96]:
new_df = new_df[['NAMEUNIT', 'Num_VC', 'Renta_Medi', 'Cuantía_m', 'Superficie', 'Num_VU', 'Renta_Me_1', 'Cuantía_1', 'Superfic_3']]
new_df = new_df.rename(columns={
    'NAMEUNIT': 'municipio', 'Num_VC': 'num_testigos_vc', 'Renta_Medi': 'renta_med_por_m2_vc', 'Cuantía_m': 'precio_mes_med_vc', 'Superficie': 'superficie_med_vc', 
    'Num_VU': 'num_testigos_vu', 'Renta_Me_1': 'renta_med_por_m2_vu', 'Cuantía_1': 'precio_mes_med_vu', 'Superfic_3': 'superficie_med_vu'
})
new_df['renta_med_total_vc'] = new_df['renta_med_por_m2_vc'] * new_df['superficie_med_vc']
new_df['renta_med_total_vu'] = new_df['renta_med_por_m2_vu'] * new_df['superficie_med_vu']

In [97]:
barna_df = new_df[new_df['municipio'].str.contains('Barcelona')]
barna_df

municipio  num_testigos_vc  renta_med_por_m2_vc  precio_mes_med_vc  \
1000  Barcelona           133528            12.570878           830.0125   

      superficie_med_vc  num_testigos_vu  renta_med_por_m2_vu  \
1000               68.0             2235            10.273973   

      precio_mes_med_vu  superficie_med_vu  renta_med_total_vc  \
1000         811.065556               98.0          854.819722   

      renta_med_total_vu  
1000         1006.849315

In [98]:
index_lloguer_df = new_df.copy()

# Filtrar por municipios con datos != 0.0
index_lloguer_df = index_lloguer_df[index_lloguer_df['renta_med_por_m2_vc'] != 0.0]

In [99]:
index_lloguer_df

municipio  num_testigos_vc  renta_med_por_m2_vc  \
4          Villamalea               36             3.139073   
7             La Roda              386             3.136059   
8              Munera               11             2.314815   
16        Madrigueras               23             2.777778   
17           Albacete             8977             5.405348   
...               ...              ...                  ...   
7583  Fuentes de Ebro               88             4.277003   
7595      Figueruelas               13             3.684211   
7602            Épila               88             4.934165   
7603            Ceuta             2147             8.787834   
7604          Melilla             2453             8.597756   

      precio_mes_med_vc  superficie_med_vc  num_testigos_vu  \
4            300.000000              108.5               56   
7            325.000000              113.0               88   
8            250.000000              105.0               17   
16           300.000000               97.0               58   
17           490.000000               93.0              339   
...                 ...                ...              ...   
7583         350.000000               87.0               74   
7595         362.582781              115.0               20   
7602         352.275336               85.0               89   
7603         650.000000               75.0              318   
7604         587.588363               69.0              502   

      renta_med_por_m2_vu  precio_mes_med_vu  superficie_med_vu  \
4                2.689551         328.916667              195.5   
7                2.544131         289.735401              201.0   
8                1.818182         270.000000              215.0   
16               2.729128         350.000000              173.0   
17               5.000000         557.136964              216.0   
...                   ...                ...                ...   
7583             2.913596         369.612869              188.5   
7595             3.053385         500.000000              173.5   
7602             3.305585         397.603486              170.0   
7603             8.108108         550.000000               66.0   
7604             6.424375         500.000000               86.0   

      renta_med_total_vc  renta_med_total_vu  
4             340.589465          525.807275  
7             354.374626          511.370403  
8             243.055555          390.909091  
16            269.444445          472.139109  
17            502.697385         1080.000000  
...                  ...                 ...  
7583          372.099303          549.212938  
7595          423.684211          529.762370  
7602          419.404027          561.949372  
7603          659.087559          535.135135  
7604          593.245192          552.496250  

[2214 rows x 11 columns]

In [101]:
# Filtrar por municipios con Barcelona en el nombre
lloguer_df = index_lloguer_df[index_lloguer_df['municipio'].str.contains('Abrera')]
lloguer_df

municipio  num_testigos_vc  renta_med_por_m2_vc  precio_mes_med_vc  \
1021    Abrera              381             7.462687              619.0   

      superficie_med_vc  num_testigos_vu  renta_med_por_m2_vu  \
1021               86.5              135             7.142857   

      precio_mes_med_vu  superficie_med_vu  renta_med_total_vc  \
1021          787.29828              164.0          645.522388   

      renta_med_total_vu  
1021         1171.428571

## 4. Model

## 5. Evaluation

## 6. Explainability